## Importing the libraries 

In [1]:
from retinaface import RetinaFace
from deepface import DeepFace
from matplotlib import pyplot as plt
import cv2
import os
import datetime as datetime
import pandas as pd
import numpy as np

In [2]:
import faiss
from datetime import datetime
import pickle

# Paths
DATABASE_DIR = 'path_to_database_images'
EMBEDDINGS_FILE = 'embeddings.pkl'
ATTENDANCE_FILE = 'attendance_log.csv'

In [3]:
embeddings = []
labels = []

DATABASE_DIR = 'students'
# Check if embeddings.pkl exists
if os.path.exists(EMBEDDINGS_FILE):
    # Load existing embeddings and labels
    with open(EMBEDDINGS_FILE, 'rb') as f:
        data = pickle.load(f)
    embeddings = data['embeddings']
    labels = data['labels']
    print("Embeddings loaded from embeddings.pkl")
else: 
    for filename in os.listdir(DATABASE_DIR):
        if filename.lower().endswith(('.png','.jpg',',jpeg')):
            label = os.path.splitext(filename)[0] # to extract the name of the filename
            img_path = os.path.join(DATABASE_DIR,filename)
            img = cv2.imread(img_path)
            
            #Detect the faces 
            faces = RetinaFace.detect_faces(img)
            if faces :
                face = faces[next(iter(faces))]
                x1,y1,x2,y2 = face['facial_area']
                face_img = img[y1:y2, x1:x2]
                
                #Extract the face embeddings from the image 
                embedding_original = DeepFace.represent(face_img, model_name='ArcFace', detector_backend="retinaface",enforce_detection= False,align = True)
                embedding_dict = embedding_original[0]
                embedding = embedding_dict["embedding"]
    
                # Convert embedding to float32 and ensure it's a 1D array
                embedding = np.array(embedding).astype('float32').flatten()
    
                embeddings.append(embedding)
                label = np.array(label)
                labels.append(label)
                print(f"{label} has been registered")
            else:
                print(f'No face detected in the {filename}')        

Embeddings loaded from embeddings.pkl


In [4]:
with open(EMBEDDINGS_FILE, 'wb') as f:
    pickle.dump({'embeddings': embeddings, 'labels': labels}, f)
print("Embeddings computed and stored in embeddings.pkl.")

Embeddings computed and stored in embeddings.pkl.


In [5]:
labels = np.array(labels)
with open(EMBEDDINGS_FILE, 'wb') as f:
    pickle.dump({'embeddings': embeddings, 'labels': labels}, f)
print("Embeddings computed and stored.")



Embeddings computed and stored.


In [6]:
# Load embeddings and labels
with open('embeddings.pkl', 'rb') as f:
    data = pickle.load(f)
database_embeddings = data['embeddings']  # Shape: (num_samples, embedding_dim)
database_labels = data['labels']
database_embeddings = np.vstack(database_embeddings)
# Dimension of the embeddings (e.g., 512 for Facenet)
dimension = database_embeddings.shape[1]

# Initialize FAISS index (Flat index for exact search)
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(database_embeddings)

print(f"Total embeddings indexed: {index.ntotal}")


Total embeddings indexed: 20


## Testing the model

In [7]:
 classroom_image = cv2.imread('classroom3.jpg')
faces = RetinaFace.detect_faces(img_path = "classroom3.jpg")
threshold = 40
for face_key in faces:
    face = faces[face_key]
    x1,y1,x2,y2 = face['facial_area']
    individual_image = classroom_image[y1:y2, x1:x2]
    individual_embedding = DeepFace.represent(img_path = individual_image , model_name='ArcFace', detector_backend="retinaface",enforce_detection= False,align = True)[0]
    individual_embedding = individual_embedding['embedding']
    individual_embedding = np.array(individual_embedding).astype('float32').reshape(1,-1)
    distance , indices = index.search(individual_embedding, k=1)
    print(distance)
    ##print(indices)
    threshold = 15
    if distance < threshold:
        print(database_labels[indices[0]])
        cv2.rectangle(classroom_image,(x1,y1),(x2,y2),(255,0,0),2)
        cv2.putText(classroom_image,str(database_labels[indices[0]]),(x1-10,y1-10),3,1.5,(255,0,0),1,cv2.LINE_AA)
        

[[12.373809]]
['Sahil Burnawal']
[[17.361927]]
[[12.261818]]
['Harish Gownatham']
[[15.32501]]
[[11.620643]]
['Haider Rizvy']
[[17.566427]]
[[15.540199]]
[[18.341595]]
[[16.411451]]
[[9.134539]]
['Ayon Binayak Ghosh']
[[12.400358]]
['Divya Raj ']
[[11.919134]]
['Shaurya']
[[13.009733]]
['Aman']


In [8]:
cv2.imwrite("output.jpg", classroom_image)

True

In [9]:
  classroom_image = cv2.imread('classroom2.jpg')
faces = RetinaFace.detect_faces(img_path = "classroom2.jpg")
threshold = 40
for face_key in faces:
    face = faces[face_key]
    x1,y1,x2,y2 = face['facial_area']
    individual_image = classroom_image[y1:y2, x1:x2]
    individual_embedding = DeepFace.represent(img_path = individual_image , model_name='ArcFace', detector_backend="retinaface",enforce_detection= False,align = True)[0]
    individual_embedding = individual_embedding['embedding']
    individual_embedding = np.array(individual_embedding).astype('float32').reshape(1,-1)
    distance , indices = index.search(individual_embedding, k=1)
    print(distance)
    print(indices)
    print(database_labels[indices[0]])
    cv2.rectangle(classroom_image,(x1,y1),(x2,y2),(255,0,0),2)
    cv2.putText(classroom_image,str(database_labels[indices[0]]),(x1-10,y1-10),3,1.5,(255,0,0),1,cv2.LINE_AA)
    

[[22.385036]]
[[4]]
['Aniket ']
[[11.351365]]
[[12]]
['Harsh Murmu']
[[16.218374]]
[[6]]
['Ankit']


In [10]:
cv2.imwrite("Output2.jpg",classroom_image)

True

In [11]:
 classroom_image = cv2.imread('classroom3.jpg')
faces = RetinaFace.detect_faces(img_path = "classroom3.jpg")
threshold = 40
for face_key in faces:
    face = faces[face_key]
    x1,y1,x2,y2 = face['facial_area']
    individual_image = classroom_image[y1:y2, x1:x2]
    individual_embedding = DeepFace.represent(img_path = individual_image , model_name='ArcFace', detector_backend="retinaface",enforce_detection= False,align = True)[0]
    individual_embedding = individual_embedding['embedding']
    individual_embedding = np.array(individual_embedding).astype('float32').reshape(1,-1)
    distance , indices = index.search(individual_embedding, k=1)
    print(distance)
    ##print(indices)
    threshold = 20
    if distance < threshold:
        print(database_labels[indices[0]])
        cv2.rectangle(classroom_image,(x1,y1),(x2,y2),(255,0,0),2)
        cv2.putText(classroom_image,str(database_labels[indices[0]]),(x1-10,y1-10),3,1.5,(255,0,0),1,cv2.LINE_AA)
        

[[12.373809]]
['Sahil Burnawal']
[[17.361927]]
['Aniket ']
[[12.261818]]
['Harish Gownatham']
[[15.32501]]
['Anish']
[[11.620643]]
['Haider Rizvy']
[[17.566427]]
['Harish Gownatham']
[[15.540199]]
['Ayon Binayak Ghosh']
[[18.341595]]
['Krishnendu Basak']
[[16.411451]]
['Basharat']
[[8.421034]]
['Ayon Binayak Ghosh']
[[12.400358]]
['Divya Raj ']
[[11.919134]]
['Shaurya']
[[13.009733]]
['Aman']


In [12]:
cv2.imwrite("output3.jpg", classroom_image)

True